In [1]:
# import dependencies
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d  # noqa: F401
import numpy as np
import pandas as pd
import seaborn as sns
import time as time

from sklearn import preprocessing

import scipy
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, ward, single

from sklearn import cluster, datasets, metrics
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from sklearn.mixture import GaussianMixture

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

!pip freeze > import_trade_data.txt

In [2]:
# enter spreadsheet name without extension:'MidCapLargeCap20240314A'
file = input("Enter Excel filename without extension: ")
# sheet = input("Enter sheet name: ")

file_path = "data/" + file + ".xlsx"

Enter Excel filename without extension: MidCapLargeCap20240314A


In [3]:
# Read the Excel file
excel_data = pd.ExcelFile(file_path)

# List all sheet names in the workbook
sheet_names = excel_data.sheet_names
print(f"Sheet Names: {sheet_names}")

# # Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
# selected_sheet_name = 'PRN_StandUpToCancer'

# # Read data from the selected sheet into a DataFrame
# df = pd.read_excel(excel_data, sheet_name=selected_sheet_name)

# # Display the DataFrame
# print(f"\nData from '{selected_sheet_name}':")
# print(df)

Sheet Names: ['Summary', 'Opinions', 'Price Performance', 'Fundamentals', 'Earnings & Dividends', 'Technicals']


In [4]:
# Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
selected_sheet_name0 = sheet_names[0]

# Read data from the selected sheet into a DataFrame
df = pd.read_excel(excel_data, sheet_name=selected_sheet_name0)

# Display the DataFrame
print(f"\nData from '{selected_sheet_name0}':")
print(df)


Data from 'Summary':
    Symbol                     Company Name  \
0      MMM                            3M Co   
1      AOS                   A O Smith Corp   
2    AMKBF     A P MOELLER MAERSK A S SHS B   
3    ABBNY                    ABB Ltd (ADR)   
4      ABT              Abbott Laboratories   
..     ...                              ...   
934    ZTS                       Zoetis Inc   
935     ZM    Zoom Video Communications Inc   
936     ZS                      Zscaler Inc   
937    ZTO   ZTO Express (Cayman) Inc (ADR)   
938  ZURVY  Zurich Insurance Group AG (ADR)   

                              Industry Prior Close 5 Day Chg 4 Week Chg  \
0         Consumer Goods Conglomerates       92.69    12.48%     -3.00%   
1    Machinery, Equipment & Components       83.69     1.96%      7.00%   
2         Freight & Logistics Services    1,357.00     1.85%    -22.00%   
3    Machinery, Equipment & Components       45.45     3.80%      6.00%   
4      Healthcare Equipment & Supplies

In [10]:
# Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
selected_sheet_name0 = sheet_names[0]

# Read data from the selected sheet into a DataFrame
df = pd.read_excel(excel_data, sheet_name = selected_sheet_name0)

# getting length of list
length = len(sheet_names)
  
# Iterate through the list to merge the data
for i in range(1, length):
    # Choose a specific sheet name (replace 'Sheet1' with the desired sheet name)
    selected_sheet_namei = sheet_names[i]
    # Read data from the selected sheet into a DataFrame
    dfi = pd.read_excel(excel_data, sheet_name = selected_sheet_namei)
    dfi = dfi.drop(['Symbol'], axis=1)
    df = pd.merge(df, dfi, on = 'Company Name', how = 'left')
#     df = pd.concat([df,dfi], ignore_index=True, axis=1)
#     print(df)
#     df_new = df.merge(dfi,how='left', left_on='Company Name', right_on='Company Name')
    

In [11]:
df

,Symbol,Company Name,Industry,Prior Close,5 Day Chg_x,4 Week Chg_x,52 Week Chg_x,Mkt Cap,Refinitiv,Argus Analyst,...,Revenue Growth CFY*,Revenue Growth NFY*,Dividend Yield,Dividend Growth 5 Year,52 Week Chg_y,Vs. 13-Day EMA*,Vs. 50-Day EMA*,% Below 52 Wk High,% Above 52 Wk Low,Short Interest as % Float
0,MMM,3M Co,Consumer Goods Conglomerates,92.69,12.48%,-3.00%,-17.00%,$51.230B,Neutral,Hold,...,1.49%,3.37%,5.80%,1.98%,-17.00%,8.56%,7.55%,8.02%,21.94%,1.33%
1,AOS,A O Smith Corp,"Machinery, Equipment & Components",83.69,1.96%,7.00%,23.00%,$10.152B,Positive,--,...,4.08%,3.97%,1.49%,9.93%,23.00%,1.98%,5.46%,1.64%,35.90%,2.44%
2,AMKBF,A P MOELLER MAERSK A S SHS B,Freight & Logistics Services,"1,357.00",1.85%,-22.00%,-43.00%,$10.126B,--,--,...,-7.43%,3.17%,5.48%,27.04%,-43.00%,-7.70%,-9.37%,42.44%,4.57%,0.00%
3,ABBNY,ABB Ltd (ADR),"Machinery, Equipment & Components",45.45,3.80%,6.00%,0.00%,$85.537B,--,--,...,4.34%,4.09%,1.92%,2.61%,0.00%,3.11%,7.58%,0.29%,51.60%,0.00%
4,ABT,Abbott Laboratories,Healthcare Equipment & Supplies,118.48,0.69%,6.00%,14.00%,$205.688B,Neutral,Buy,...,4.74%,7.23%,1.83%,12.74%,14.00%,0.64%,4.38%,1.22%,34.00%,0.87%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,ZTS,Zoetis Inc,Pharmaceuticals,185.23,-3.98%,1.00%,12.00%,$84.811B,Neutral,Buy,...,7.52%,7.10%,0.98%,24.37%,12.00%,-5.19%,-7.14%,12.72%,16.69%,0.83%
935,ZM,Zoom Video Communications Inc,Software & IT Services,66.49,3.04%,10.00%,0.00%,$17.347B,Positive,Hold,...,0.00%,0.00%,0.00%,0.00%,0.00%,1.00%,2.19%,9.83%,16.26%,4.98%
936,ZS,Zscaler Inc,Software & IT Services,206.36,-0.49%,-10.00%,84.00%,$30.610B,Neutral,--,...,31.31%,25.47%,0.00%,0.00%,84.00%,-5.45%,-10.44%,22.41%,137.17%,4.65%
937,ZTO,ZTO Express (Cayman) Inc (ADR),Freight & Logistics Services,20.05,6.02%,26.00%,-18.00%,$12.166B,--,--,...,9.37%,13.45%,0.00%,0.00%,-18.00%,7.15%,10.47%,27.89%,36.33%,3.48%
